In [1]:
# reloads modules when running again
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from model_functions import *

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import pairwise_distances

import warnings
warnings.filterwarnings('ignore')

## Data loading

In [3]:
data, metadata = load_data()

### Constants

In [4]:
test_size = 10

## Data preprocessing

### Split to train-test

In [5]:
# sort data by collection date
data = data.sort_values(by="collection_date")

# split to train & test by test_size last samples
test_df = data.groupby('baboon_id').tail(test_size)
train_df = data.drop(test_df.index)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# can be uncomment and used for extrapulation
# train_df = data.groupby('baboon_id').head(2)
# test_df = data.drop(train_df.index)
# test_df = test_df.reset_index(drop=True)
# train_df = train_df.reset_index(drop=True)

In [6]:
meta_features_union = meta_features + ["month_sin", "month_cos"]
x_test, y_test = test_df[meta_features_union], test_df.drop(columns=meta_features_union)
x_test["interpolated"] = 0

### Interpolation

In [7]:
# aggregate to one sample per week
train_df = aggregate_samples(train_df.copy())

In [ ]:
interpolated_df = knn_interpolation(train_df)
interpolated_df.to_csv("interpolated_df.csv")

finished interpolating baboon 1
finished interpolating baboon 2
finished interpolating baboon 3
finished interpolating baboon 4
finished interpolating baboon 5
finished interpolating baboon 6
finished interpolating baboon 7
finished interpolating baboon 8
finished interpolating baboon 9
finished interpolating baboon 10


In [ ]:
# can be uncomment for loading the data from a csv file instead of running the knn_interpolation function

# interpolated_df = pd.read_csv("interpolated_df.csv")

In [ ]:
interpolated_df["collection_date"] = pd.to_datetime(interpolated_df["collection_date"])
taxa_columns = [col for col in interpolated_df.columns if col not in meta_features_union + ["interpolated"]]

## Train

In [ ]:
# can be uncomment and used to run only the trend model

# pred_df = trend_pred(train_df, x_test)
# pred_df.to_csv("trend_df.csv")

In [ ]:
# can be uncomment and used to run only the seasonal model

# pred_df = seasonal_pred(train_df, x_test)
# pred_df.to_csv("seasonal_df.csv")

In [ ]:
pred_df = predict(train_df, x_test)
pred_df.to_csv("pred_df.csv")

## Performance analysis

In [ ]:
# can be uncomment for loading the data from a csv file instead of running the predict function

# pred_df = pd.read_csv("pred_df.csv")

In [ ]:
# calculating Bray-Curtis score
d_matrix = pairwise_distances(pred_df[y_test.columns], y_test, metric="braycurtis").diagonal()
pd.DataFrame(d_matrix).to_csv("braycurtis_performance_2_time_points.csv")
braycurtis_score = d_matrix.mean()

### Performance Analysis

In [ ]:
print(f"Bray-Curtis score: {round(braycurtis_score, 5)}")

In [ ]:
analysis_df = x_test.copy()
analysis_df["braycurtis_score"] = d_matrix
analysis_df.drop(columns=["interpolated", "month_sin", "month_cos"], inplace=True)

In [ ]:
# Bray-Curtis Distribution
plt.grid(False)
sns.histplot(analysis_df["braycurtis_score"], color="#46c1db", bins=25, kde=True)
plt.title("Bray-Curtis Distribution")
plt.xlabel("Bray-Curtis Score")
plt.xlim(0)
plt.show()

In [ ]:
# Bray-Curtis Distribution
plt.grid(False)
sns.histplot(analysis_df.groupby("baboon_id")[["braycurtis_score"]].agg("mean")["braycurtis_score"], color="#46c1db", bins=25, kde=True)
plt.title("Average Bray-Curtis per Baboon")
plt.xlabel("Average per Baboon")
plt.show()

In [ ]:
# Bray-Curtis by sex
plt.grid(False)
sns.violinplot(data=analysis_df, hue="sex", y="braycurtis_score", palette="muted")
plt.title("Bray-Curtis per Sex")
plt.xlabel("Sex")
plt.ylabel("Bray-Curtis Score")
plt.show()

In [ ]:
# Bray-Curtis per Baboon
plt.figure(figsize=(30, 6))
plt.grid(True)
analysis_df["baboon_mean"] = analysis_df.groupby("baboon_id")[["braycurtis_score"]].transform("median")["braycurtis_score"]
sns.violinplot(data=analysis_df.sort_values("baboon_mean"), hue="baboon_id", y="braycurtis_score", palette="muted", legend=False)
plt.title("Bray-Curtis per Baboon")
plt.xlabel("Baboon")
plt.xlim((-0.41,0.41))
plt.ylabel("Bray-Curtis Score")
plt.show()

In [ ]:
# creating a dictionary of babbon_id:last known sample
d = dict()
for baboon_id in train_df["baboon_id"].unique():
    d[baboon_id] = [sorted(train_df[train_df["baboon_id"] == baboon_id]["collection_date"].to_list())[-1]]
df1 = pd.DataFrame(d).T
merged_df = analysis_df.merge(df1, left_on="baboon_id", right_index=True)

In [ ]:
# computing the distance between each sample to the last one in the train dataset
merged_df.rename(columns={0:"last_timepoint"}, inplace=True)
merged_df["dist_in_days"] = 0
merged_df["dist_in_days"] = merged_df.apply(lambda row: np.abs(row["collection_date"] - row["last_timepoint"]).total_seconds() / (60 * 60 * 24), axis=1)

In [ ]:
# plot the Bray-Curtis score for the distances in days
mean_by_dist = merged_df.groupby("dist_in_days")[["braycurtis_score"]].mean()
plt.figure(figsize=(30, 6))
sns.regplot(x=mean_by_dist.index, y=mean_by_dist["braycurtis_score"], color="#46c1db")
plt.xlabel("Distance From the Last Known Sample (Days)")
plt.title("Bray-Curtis Score by Distance in Days From the Last Known Sample")
plt.show()